In [1]:
import cv2
import numpy as np

In [2]:
# image_path = 'images/receipt2.jpeg'
image_path = 'images/receipt_image.jpg'

In [3]:
image = cv2.imread(image_path, 0)
color = cv2.imread(image_path)

hsv = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)

hue, saturation, value = cv2.split(hsv)

In [4]:
def show(image):
    
    resized_height = 720
    percent = resized_height / len(image)
    resized_width = int(percent * len(image[0]))
    gray = cv2.resize(image,(resized_width,resized_height))


    cv2.imshow('cringe', gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
if np.mean(color) < 128:
    thresh = cv2.threshold(hue, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
else:
    thresh = cv2.threshold(hue, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

In [6]:
eroded = cv2.erode(thresh, kernel=None, iterations=7)
dilated = cv2.dilate(eroded, kernel=None, iterations=15)
eroded = cv2.erode(dilated, kernel=None, iterations=100)
dilated = cv2.dilate(eroded, kernel=None, iterations=100)

In [7]:
show(dilated)

In [8]:
def extract_largest_rectangular_contour(color_image, preprocessed_image):
    color_image = color_image.copy()
    contours, _ = cv2.findContours(preprocessed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = None
    largest_area = 0
    largest_approx = None
    
    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        if len(approx) == 4:
            area = cv2.contourArea(contour)
            if area > largest_area:
                largest_area = area
                largest_contour = contour
                largest_approx = approx
                
    if largest_contour is not None:
        cv2.drawContours(color_image, [largest_approx], -1, (0, 255, 0), 3)
        x, y, w, h = cv2.boundingRect(largest_contour)
        cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        return color_image, largest_approx
    
    return color_image, None

In [9]:
im, cor = extract_largest_rectangular_contour(color, dilated)
show(im)

In [10]:
top = max(cor[0][0][1], cor[3][0][1])
bottom = min(cor[1][0][1], cor[2][0][1])
left = max(cor[0][0][0], cor[1][0][0])
right = min(cor[2][0][0], cor[3][0][0])
cropped = image[top:bottom, left:right]
show(cropped)

In [11]:
def order_points(coords):
    pts = coords - [left, top]
    pts = pts.reshape(4, 2)
    sum_pts = pts.sum(axis=1)
    diff_pts = np.diff(pts, axis=1)
    
    top_left = pts[np.argmin(sum_pts)]
    top_right = pts[np.argmin(diff_pts)]
    bottom_right = pts[np.argmax(sum_pts)]
    bottom_left = pts[np.argmax(diff_pts)]
    
    return np.array([top_left, top_right, bottom_right, bottom_left])

def calculate_size(img, ordered):
    width = int(img.shape[1] * 0.8)
    aspect_ratio = np.linalg.norm(ordered[0] - ordered[3]) / np.linalg.norm(ordered[0] - ordered[1])
    return width, int(width * aspect_ratio)

def process_image(img, contour):
    ordered = order_points(contour)
    img_with_pts = img.copy()
    for pt in ordered:
        img_with_pts = cv2.circle(img_with_pts, tuple(pt), 10, (255, 255, 255), -1)
  
    w, h = calculate_size(img, ordered)
    
    pts1 = np.float32(ordered)
    pts2 = np.float32([[0, 0], [w, 0], [w, h], [0, h]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    
    corrected_img = cv2.warpPerspective(img, matrix, (w, h))
    
    return img_with_pts, corrected_img

In [64]:
_, corrected = process_image(cropped, cor)
show(corrected)

In [13]:
def unsharp_masking(image, k=1):
    blur = cv2.GaussianBlur(image, (11,11), 0)
    return cv2.addWeighted(image, k+1, blur, -k, 1)

In [65]:
unsharp = unsharp_masking(corrected, 11)
show(unsharp)

In [223]:
thresh = np.where(unsharp > 64, 255, 0).astype(np.uint8)
show(thresh)

In [204]:
thresh_mask = cv2.dilate(thresh, kernel=np.ones((3,6)), iterations=1)
show(thresh_mask)

In [188]:
hist_eq = cv2.equalizeHist(np.uint8(np.power(corrected / 255.0, 0.5) * 255))

In [203]:
test = cv2.bitwise_and(corrected, corrected, mask=thresh_mask)
show(test)
show(np.where(test > 100, 0, 255).astype(np.uint8))

In [212]:
show(test)

In [ ]:
# thresholding, 

In [ ]:
show()

In [234]:
test2 = cv2.adaptiveThreshold(test,255,cv2.CALIB_CB_ADAPTIVE_THRESH,cv2.THRESH_BINARY,3,2)

In [235]:
show(cv2.erode(test2, kernel=None))